<a href="https://colab.research.google.com/github/Yyssjj96/__practice___/blob/main/%EB%A1%9C%EA%B7%B8%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%99%9C%EC%9A%A9(%ED%8D%BC%EB%84%90%EB%B6%84%EC%84%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('/content/drive/MyDrive/ecommerce_behavior.csv')

In [4]:
df.head(10)
df.drop('Unnamed: 0',axis=1,inplace=True)

- event_time : 이벤트가 발생한 시간
- event_type : 이벤트 종류
  - view : 상품조회
  - cart : 카트에 상품 추가
  - remove_from_cart : 카트 상품 제거
  - purchase : 구매
- product_id : 상품번호
- category_id : 카테고리 번호
- category_brand : 카테고리명
- brand : 브랜드명
- price : 가격
- user_id, user_session : 유저아이디, 유저세션

질문
- DAU(일간 활성 사용자수) 추이는 ?
  - 어느 요일에 가장 많이 방문하는가 ?
- 사이트 체류시간 평균은 ?
  - 조회만 한 유저, 카트에 담은 유저, 구매까지 한 유저별로 체류시간이 어떻게 다른가
- 퍼널 분석
  - 어느 단체에서 유저들이 가장 많이 이탈하는가?
  

In [8]:
# 타입 변경
df['event_time'] = pd.to_datetime(df['event_time'],format='%Y-%m-%d %H:%M:%S UTC')

In [12]:
df.isnull().sum()
# null 값 삭제
df.drop(['category_code','brand'],axis=1,inplace=True)

In [15]:
df['date_ymd'] = df['event_time'].dt.date
df['date_ymd'] = pd.to_datetime(df['date_ymd'],format ='%Y-%m-%d' )

In [18]:
# dau(일간 활성 사용자 수) 추이  -- 어느 요일에 가장 많이 방문할까
dau = df.groupby('date_ymd')[['user_id']].nunique().reset_index().rename({'user_id':'dau'},axis=1)

In [22]:
dau
fig = px.line(data_frame=dau,x='date_ymd',y='dau',title='dau 추이')
fig.show()

In [24]:
dau['day_of_week'] = dau['date_ymd'].dt.day_name()
dau['day_of_week1'] = dau['date_ymd'].dt.day_of_week

In [26]:
avg_dau_by_dow = dau.groupby(['day_of_week','day_of_week1'])[['dau']].mean().reset_index()
avg_dau_by_dow.sort_values('day_of_week1',inplace=True)

In [28]:
avg_dau_by_dow
fig = px.bar(data_frame=avg_dau_by_dow,x='day_of_week',y='dau',title='평균 dau')
fig.show()
# 화욜에 제일 높고 보통 평일이 주말보다 더 높다.

In [34]:
# 체류시간 구하기 : 체류시간은 한 세션의 끝에서 시작시간을 뺀 값으로 정의
max_ = df.query('user_session == "2806ff10-08bc-4811-9ab7-af074fe22a88"')['event_time'].max()
min_ = df.query('user_session == "2806ff10-08bc-4811-9ab7-af074fe22a88"')['event_time'].min()
temp_time = max_ - min_

In [36]:
duration = df.groupby('user_session')[['event_time']].agg(['max','min']).reset_index()

In [37]:
duration['duration'] = duration['event_time']['max'] - duration['event_time']['min']
duration.columns = ['user_session','max','min','duration']

In [39]:
duration['duration'].mean()
# 평균 체류시간은 한시간

Timedelta('0 days 00:59:16.683693260')

In [41]:
# 이벤트 별로 체류시간을 살펴보자
session_pivot = pd.pivot_table(data=df, index='user_session',columns='event_type',values='event_time',aggfunc='count').reset_index().fillna(0)
session_pivot

event_type,user_session,cart,purchase,remove_from_cart,view
0,0000061d-f3e9-484b-8c73-e54f355032a3,0.0,0.0,0.0,1.0
1,00000ac8-0015-4f12-996a-be2896323738,0.0,0.0,0.0,1.0
2,00001ca1-f2df-4572-b0b8-e752e2064aae,0.0,0.0,0.0,1.0
3,00002db7-16b6-4db2-bf8b-7a1cb6bd0e7f,0.0,0.0,0.0,1.0
4,00002f68-09b8-4db3-a092-aeff45fd13ad,0.0,0.0,0.0,1.0
...,...,...,...,...,...
911569,ffff7b96-9751-4eaa-806e-fe979cc00dc8,1.0,0.0,2.0,10.0
911570,ffff80e2-ad33-4704-9ffe-d6c612e9641f,0.0,0.0,0.0,1.0
911571,ffff8da3-b79a-48f2-888c-117f2d1a7793,0.0,0.0,0.0,1.0
911572,ffff9422-39ba-4cdf-afd1-a9d87bb3d79b,0.0,0.0,0.0,1.0


In [43]:
cart_session = list(session_pivot.query('cart > 0')['user_session'])
purchase_session = list(session_pivot.query('purchase > 0')['user_session'])

In [45]:
# 조회만 한 유저의 평균 체류시간
duration.query('user_session not in @cart_session and user_session not in @purchase_session')['duration'].mean()

Timedelta('0 days 00:38:30.953374025')

In [46]:
view_session_avg_duration = duration.query('user_session not in @cart_session and user_session not in @purchase_session')['duration'].mean()
cart_session_avg_duration = duration.query('user_session in @cart_session')['duration'].mean()
purchase_session_avg_duration = duration.query('user_session in @purchase_session')['duration'].mean()

In [47]:
print(f'조회만 한 유저의 평균 체류시간 : {view_session_avg_duration}')
print(f'카트에 담은 유저의 평균 체류시간 : {cart_session_avg_duration}')
print(f'구매한 유저의 평균 체류시간 : {purchase_session_avg_duration}')

조회만 한 유저의 평균 체류시간 : 0 days 00:38:30.953374025
카트에 담은 유저의 평균 체류시간 : 0 days 02:39:48.642760643
구매한 유저의 평균 체류시간 : 0 days 06:42:21.679333566


퍼널 분석 : 흐름으로 이탈률 보기

In [48]:
funnel = session_pivot[['view','cart','remove_from_cart','purchase']].sum().to_frame().reset_index()
funnel.columns = ['event_type','count']
funnel = funnel.query('event_type != "remove_from_cart"')
funnel

,event_type,count
0,view,2035188.0
1,cart,957169.0
3,purchase,184619.0


In [49]:
fig = px.funnel(data_frame=funnel,x='count',y='event_type')
fig.update_traces(texttemplate='%{value:,.of}')
fig.show()

In [50]:
# 단계별 이탈율 계산
view_to_cart = list(funnel['count'])[1] / list(funnel['count'])[0]
view_to_purchase = list(funnel['count'])[2] / list(funnel['count'])[0]

In [51]:
funnel['retain_rate'] = [1, view_to_cart, view_to_purchase]
funnel

,event_type,count,retain_rate
0,view,2035188.0,1.000000
1,cart,957169.0,0.470310
3,purchase,184619.0,0.090713


In [54]:
fig = px.funnel(data_frame=funnel,x='event_type',y='retain_rate')
fig.update_traces(texttemplate='%{value:,.2%}')
fig.show()
# 장바구니는 전체에 47, 구매까지는 9%